# Distinguishing Adult and Youth Faces Using Convolutional Neural Networks


## Notebook CNN2: Second Convolutional Neural Network
This section details the development of an advanced Convolutional Neural Network (CNN) incorporating two hidden layers. In this model, the complexity is increased to capture more intricate patterns in the data. However, neither regularization techniques nor data balancing methods are applied at this stage. This iteration aims to assess the impact of additional layers on model performance. Future iterations will introduce parameter adjustments, including regularization and data balancing, to progressively enhance the model's effectiveness and mitigate overfitting.

### Important Considerations
* These models require significant computing power. Each took 10 hours to fit using an M3 chip and 18GB of memory.
* Consider making a separate keras environment
* Consider working in GoogleColab


In [65]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.utils import compute_class_weight

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

import keras
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

In [66]:
data_dir = '/Users/marta/Documents/data_dir/'

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(256, 256),
  batch_size = 32)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(256, 256),
  batch_size = 32)

Found 14633 files belonging to 2 classes.
Using 11707 files for training.
Found 14633 files belonging to 2 classes.
Using 2926 files for validation.


In [ ]:
# Creating weights to address the unbalanced classes.
# Assisted by Argo Ovsepyan
classes = np.array(['PLP', 'POR'])
y = [classes[0]] * 5422 + [classes[1]] * 9211
class_weights = compute_class_weight('balanced', classes=classes, y=y)
class_weights_dict = class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
# CNN 2: input layer + 2 hidden layers, changed filters to 3 and kernel size to 3
cnn2 = Sequential()

# Convoluting and MaxPooling
# input (which includes one hidden layer because Sequential does that)
cnn2.add(Conv2D(512, 3, # changed from 2
                activation = 'relu',
                input_shape = (256, 256, 3)))
cnn2.add(MaxPooling2D(2, padding = 'same'))

# add our first explicit hidden layer
cnn2.add(Conv2D(256, 
                3, 
                activation = 'relu',
               kernel_regularizer = regularizers.l2(0.01)))
cnn2.add(MaxPooling2D(2, padding = 'same'))

# adding a second hidden layer
cnn2.add(Conv2D(256, 
                3,
                activation = 'relu',
               kernel_regularizer = regularizers.l2(0.01)))
cnn2.add(MaxPooling2D(2, padding = 'same'))

# Output layer, with softmax activation because it's classification
# with as many neurons as there are classes
cnn2.add(Flatten())
cnn2.add(Dense(1, activation = 'sigmoid'))

# compiling the model
cnn2.compile(
    loss = 'binary_crossentropy', 
    optimizer = 'adam', 
    metrics = ['acc'])

 # Fit the model
 history2 = cnn2.fit(train_ds,
                    epochs = 10,
                    validation_data = val_ds, 
                    class_weight = class_weights_dict)

In [ ]:
cnn2.save('./saved_models/cnn2.h5')
pd.DataFrame(history2.history).to_csv('./saved_models//history2.csv', index=False)

In [ ]:
# Check out the plot of loss vs epoch.
plt.figure(figsize = (12, 6));

plt.subplot(1,2,1)
plt.plot(history2.history['loss'], c = 'navy', label = 'Training Loss');
plt.plot(history2.history['val_loss'], c = 'orange', label = 'Testing Loss');

plt.title('''CNN 2 :
Binary Crossentropy (loss function),
as a Function of Epochs''')
plt.xlabel('Epochs');
plt.ylabel('Loss Function')
plt.legend();


plt.subplot(1, 2, 2)
plt.plot(history2.history['acc'], c = 'navy', label = 'Training Accuracy');
plt.plot(history2.history['val_acc'], c = 'orange', label = 'Testing Accuracy');
plt.title('''CNN 2: 
Accuracy Score 
as a Function of Epochs)''')
plt.xlabel('Epochs');
plt.ylabel('Accuracy Score')
plt.legend();

 **CNN 2**: By adding more hidden layers, CNN 2 significantly improved training accuracy. However, the model showed signs of overfitting, as indicated by a lower validation accuracy compared to training accuracy. The lack of regularization meant that while the model could learn from the training data, it struggled to generalize to new, unseen data.